In [1]:
import numpy as np
import numpy.random as npr
import math
from scipy import stats
import datetime as dt
import random
import matplotlib.pyplot as plt

In [2]:
np.random.seed(100)

In [3]:
r = 0.05
S_0 = 1.22
T = 1
K = 1.22
typeCP = 'call'

In [4]:
N = 100000
mu, sigma = 0, 0.1 # mean and standard deviations

In [5]:
def EU_montecarlo(T,N,S_0,r,sigma,K, typeCP):
    typeCP = 1 if typeCP == "call" else -1    
    S = []
    P = []
    for n in range(N):
        Z = np.random.normal(mu, sigma, 1)
        S.append(S_0*np.exp((r - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z))
        h = np.maximum(typeCP*(S[-1] - K), 0)
        P.append(math.exp(-r*T)*h)
    P_hat = np.mean(P)
    return P_hat

In [6]:
#taken from Optiver TraderCraft workshop and competition

_norm_cdf = stats.norm(0, 1).cdf
_norm_pdf = stats.norm(0, 1).pdf


def _d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


def _d2(S, K, T, r, sigma):
    return _d1(S, K, T, r, sigma) - sigma * np.sqrt(T)


def calculate_time_to_date(expiry_datetime: dt.datetime, current_datetime: dt.datetime):
    """
    Calculate the time until expiration of the option. All options expire at 12:00:00 UTC on the date of expiry.
    
    Example: calculate_time_to_date(dt.datetime.strptime('2021-03-01 12:00:00', '%Y-%m-%d %H:%M:%S'), dt.datetime.now())
    """
    return (expiry_datetime - current_datetime) / dt.timedelta(days=1) / 365


def call_value(S, K, T, r, sigma):
    """
    The fair value of a call option paying max(S-K, 0) at expiry, under the Black-scholes model,
    for an option with strike <K>, expiring in <T> years, under a fixed interest rate <r>,
    a stock volatility <sigma>, and when the current price of the underlying stock is <S>.

    Parameters
    ----------
    S : float
        The current value of the underlying stock.

    K : float
        The strike price of the option.

    T : float
        Time to expiry in years.

    r : float
        The fixed interest rate valid between now and expiry.

    sigma : float
        The volatility of the underlying stock process.

    Returns
    -------
    call_value : float
        The fair present value of the option.
    """

    return S * _norm_cdf(_d1(S, K, T, r, sigma)) - K * np.exp(-r * T) * _norm_cdf(
        _d2(S, K, T, r, sigma)
    )


def put_value(S, K, T, r, sigma):
    """
    The fair value of a put option paying max(K-S, 0) at expiry, under the Black-scholes model,
    for an option with strike <K>, expiring in <T> years, under a fixed interest rate <r>,
    a stock volatility <sigma>, and when the current price of the underlying stock is <S>.

    Parameters
    ----------
    S : float
        The value of the underlying stock.

    K : float
        The strike price of the option.

    T : float
        Time to expiry in years.

    r : float
        The fixed interest rate valid between now and expiry.

    sigma : float
        The volatility of the underlying stock process.

    Returns
    -------
    put_value : float
        The fair present value of the option.
    """

    return np.exp(-r * T) * K * _norm_cdf(-_d2(S, K, T, r, sigma)) - S * _norm_cdf(
        -_d1(S, K, T, r, sigma)
    )

In [7]:
call_bs = call_value(S_0, K, T, r, sigma)

In [8]:
put_bs = put_value(S_0, K, T, r, sigma)

In [9]:
call_mc = EU_montecarlo(T,N,S_0,r,sigma,K, 'call')

In [10]:
put_mc = EU_montecarlo(T,N,S_0,r,sigma,K, 'put')

In [11]:
print(call_bs, call_mc)
print(put_bs,put_mc)
e_call = (call_bs-call_mc)/call_bs
print(e_call*100,'%')
e_put = (put_bs-put_mc)/put_bs
print(e_put*100,'%')

0.08302048404763018 0.05350914424637699
0.02352038193850131 0.0
35.547058222790014 %
100.0 %


In [18]:
#Asian call
s = [S_0]
c = []
n = 1000
m = 5

for i in range(n):
    for j in range(m):
        z = np.random.normal(mu, sigma, 1)
        s.append(s[-1]*math.exp((r-0.5*sigma**2)*(T/m) + sigma*math.sqrt(T/m)*z))
    s_bar = np.mean(s)
    s = [S_0]
    c.append(math.exp(-r*T)*max(0, s_bar-K))
c_hat = np.mean(c)

In [13]:
def lsm_option(T, N, steps, S, r, sigma, K, option_type):

    if option_type == "call":
        option_type = 1
    else:
        option_type = -1
        
    dt = np.float(T) / steps
    df = np.exp(-r * dt)
    
    # simulation of index levels
    sample_path = np.zeros((steps + 1, N))
    sample_path[0] = S
    z = npr.randn(steps, N)
    for t in range(1, steps + 1):
        sample_path[t] = sample_path[t-1]*np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z[t-1])

    # payoff computation
    h = np.maximum(option_type*(sample_path - K), 0)
    # LSM algorithm - backward recursion from final node
    V = np.copy(h)
    for t in range(steps-1, 0, -1):
        reg = np.polyfit(sample_path[t], V[t+1]*df, 10) #compute beta_i using second order polynomial fit
        C = np.polyval(reg, sample_path[t]) #compute 
        V[t] = np.where(C > h[t], V[t+1]*df, h[t])
    # MCS estimator
    option = df * 1 / N * np.sum(V[1])
    return option

In [14]:
T = 1
N = 100000
steps = 12
S = 1.22
r = 0
sigma = 0.1
K = 1.22
option_type = 'put'

In [15]:
lsm_option(T, N, steps, S, r, sigma, K, option_type)

<ipython-input-15-5edda9cf4515>:1: RankWarning: Polyfit may be poorly conditioned
  lsm_option(T, N, steps, S, r, sigma, K, option_type)
<ipython-input-15-5edda9cf4515>:1: RankWarning: Polyfit may be poorly conditioned
  lsm_option(T, N, steps, S, r, sigma, K, option_type)
<ipython-input-15-5edda9cf4515>:1: RankWarning: Polyfit may be poorly conditioned
  lsm_option(T, N, steps, S, r, sigma, K, option_type)
<ipython-input-15-5edda9cf4515>:1: RankWarning: Polyfit may be poorly conditioned
  lsm_option(T, N, steps, S, r, sigma, K, option_type)
<ipython-input-15-5edda9cf4515>:1: RankWarning: Polyfit may be poorly conditioned
  lsm_option(T, N, steps, S, r, sigma, K, option_type)
<ipython-input-15-5edda9cf4515>:1: RankWarning: Polyfit may be poorly conditioned
  lsm_option(T, N, steps, S, r, sigma, K, option_type)
<ipython-input-15-5edda9cf4515>:1: RankWarning: Polyfit may be poorly conditioned
  lsm_option(T, N, steps, S, r, sigma, K, option_type)
<ipython-input-15-5edda9cf4515>:1: RankWa

0.04883275289324841

In [16]:
print(call_bs, call_mc)
print(put_bs,put_mc)
e_call = (call_bs-call_mc)/call_bs
print(e_call*100,'%')
e_put = (put_bs-put_mc)/put_bs
print(e_put*100,'%')

0.08302048404763018 0.05350914424637699
0.02352038193850131 0.0
35.547058222790014 %
100.0 %
